In [ ]:
!pip install transformers

In [ ]:
import tensorflow as tf
import torch

# 获取GPU名字
device_name = tf.test.gpu_device_name()

if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' %torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
  #!nvidia-smi
else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!pip install wget

In [ ]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2021-01-23 04:48:35--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  24.3MB/s    in 3.3s    

2021-01-23 04:48:39 (24.3 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
!tar -zxvf aclImdb_v1.tar.gz

In [ ]:
import re
import os
import pandas as pd

# Load all files from directory in a dataframe
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge pos and neg data, add a polarity column and shuffle
def load_data(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)# sample开始的后面用于随机打乱dataframe的数据

train = load_data(os.path.join(os.path.dirname('aclImdb_v1.tar.gz'), "aclImdb", "train"))
test = load_data(os.path.join(os.path.dirname('aclImdb_v1.tar.gz'), "aclImdb", "test"))

In [ ]:
train = train.sample(10000)
test = test.sample(5000)

In [ ]:
train

,sentence,sentiment,polarity
22441,My wife and I took our 13 year old son to see ...,10,1
18494,While William Shater can always make me smile ...,1,0
12093,"Ever since seeing this film as a child, over 3...",9,1
22438,This is just a bad movie. With what seemed to ...,2,0
6790,Immediately after renting and watching this mo...,1,0
...,...,...,...
13752,Just another film that exploits gratuitous fro...,1,0
11197,This great movie has failed to register a high...,10,1
143,...but the general moviegoer's mileage may var...,8,1
24879,I was pleasantly surprised by the film. Let's ...,7,1


In [ ]:
# Get the lists of sentences and labels
sentences = train.sentence.values
labels = train.polarity.values

In [ ]:
from transformers import BertTokenizer

# Load the Bert tokenizer
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [ ]:
# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = []

for sent in sentences:
  encode_dict = tokenizer.encode_plus(
                      sent,
                      add_special_tokens = True,
                      max_length = 128,
                      pad_to_max_length = True,
                      return_attention_mask = True,
                      return_tensors = 'pt',  
                )
  input_ids.append(encode_dict['input_ids'])
  attention_masks.append(encode_dict['attention_mask'])

# Convert the lists to tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

print('original:', sentences[0])
print('Token IDs:', input_ids[0])

In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Combine the training inputs into a TensorDataset
dataset = TensorDataset(input_ids, attention_masks, labels)

# 90% for training and 10% for validation
train_size = int(0.9* len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 32
# Create the dataloaders for training and validation sets
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )
validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)
model.cuda()

optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                  )

epochs = 4
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np


# define a function for calculating accuracy
def flat_accuracy(preds, labels):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat)/ len(labels_flat)

In [ ]:
import time
import datetime

# 一个把运行时间格式化为 hh:mm:ss
def format_time(elapsed):
  # 舍入到最接近的秒
  elapsed_rounded = int(round((elapsed)))
  # Format as hh:mm:ss
  return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# starting training
import random
import numpy as np
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# We'll store a number of quantities such as training and validation loss,
# validation accuracy, and timings.
training_stats = []
# Measure the total training time for the whole run.
total_t0 = time.time()
# For each epoch...
for epoch_i in range(0, epochs):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    # Reset the total loss for this epoch.
    total_train_loss = 0
    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()
        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        outputs = model(b_input_ids,
                              token_type_ids=None,
                              attention_mask=b_input_mask,
                              labels=b_labels)
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        loss = outputs[0]
        logits = outputs[1]
        total_train_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()
    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            outputs = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]
        # Accumulate the validation loss.
        total_eval_loss += loss.item()
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    282.    Elapsed: 0:00:28.
  Batch    80  of    282.    Elapsed: 0:00:57.
  Batch   120  of    282.    Elapsed: 0:01:25.
  Batch   160  of    282.    Elapsed: 0:01:53.
  Batch   200  of    282.    Elapsed: 0:02:21.
  Batch   240  of    282.    Elapsed: 0:02:49.
  Batch   280  of    282.    Elapsed: 0:03:18.

  Average training loss: 0.16
  Training epcoh took: 0:03:18

Running Validation...
  Accuracy: 0.87
  Validation Loss: 0.44
  Validation took: 0:00:07

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    282.    Elapsed: 0:00:28.
  Batch    80  of    282.    Elapsed: 0:00:57.
  Batch   120  of    282.    Elapsed: 0:01:25.
  Batch   160  of    282.    Elapsed: 0:01:53.
  Batch   200  of    282.    Elapsed: 0:02:21.
  Batch   240  of    282.    Elapsed: 0:02:50.
  Batch   280  of    282.    Elapsed: 0:03:18.

  Average training loss: 0.14
  Training epcoh took: 0:03:19

Running Validation...
  Accuracy: 0.87
 

In [ ]:
import pandas as pd

# 保留两位小数
pd.set_option('precision', 2)

df_stats = pd.DataFrame(data=training_stats)
# use the epoch as the row index
df_stats = df_stats.set_index('epoch')

df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.16,0.44,0.87,0:03:18,0:00:07
2,0.14,0.41,0.87,0:03:19,0:00:07
3,0.08,0.46,0.88,0:03:18,0:00:07
4,0.06,0.46,0.88,0:03:18,0:00:07


可以看到training loss越来越小但是validation loss 却在增加，说明model的训练时间过长，有过拟合训练数据的现象存在。

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

sns.set(style='darkgrid')
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)
# 绘制曲线
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1,2,3,4])

plt.show()

# Test
Test Data Preparation


In [ ]:
# Create sentence and label lists like training
sentences = test.sentence.values
labels = test.polarity.values

# Tokenize all of the sentences and map the tokens to their word IDs
input_ids = []
attention_masks = []

for sent in sentences:
  encode_dict = tokenizer.encode_plus(
                      sent,
                      add_special_tokens = True,
                      max_length = 128,
                      pad_to_max_length = True,
                      return_attention_mask = True,
                      return_tensors = 'pt', 
              )
  input_ids.append(encode_dict['input_ids'])
  attention_masks.append(encode_dict['attention_mask'])

# Covert the lists to tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# set batch size
batch_size = 32

# Create the Dataloader
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# start testing
print('Start prediction')

# Put model in evaluation mode
model.eval()

eval_acc, eval_step = 0,0

# Predict
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)

  b_input_ids, b_input_mask, b_labels = batch

  # 告诉model 不要计算和保存梯度，节省内存，加快预测速度
  with torch.no_grad():
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
  logits = outputs[0]

  # Move logits and labels to Cpu
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  tmp_eval_acc = flat_accuracy(logits, label_ids)
  eval_acc += tmp_eval_acc
  eval_step +=1

eval_acc = eval_acc/eval_step
print('Accuracy:', eval_acc)
print('    Done')

Start prediction
Accuracy: 0.8765923566878981
    Done
